In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

## Get Devic for Training

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [3]:
class NerualNetwork(nn.Module):
    def __init__(self):
        super(NerualNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )
        
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [4]:
model = NerualNetwork().to(device)
print(model)

NerualNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [7]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([2], device='cuda:0')


## Model Layers

In [6]:
input_image = torch.rand(3, 28, 28)
print(input_image.size())

torch.Size([3, 28, 28])


In [8]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


In [9]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


In [10]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[-0.2973,  0.2466, -0.4476,  0.1987, -0.3091, -0.2210,  0.4755,  0.0041,
          0.3738,  0.0953, -0.6307, -0.0699,  0.0259, -0.0969, -0.4417,  0.1725,
          0.2422, -0.0033,  0.2988, -0.1051],
        [-0.4104,  0.3139, -0.3696,  0.2604, -0.3481, -0.6282,  0.5266,  0.1123,
          0.6165,  0.1251, -0.3387, -0.1553,  0.1692, -0.4765, -0.3053,  0.2201,
          0.2858,  0.1468,  0.1331, -0.0753],
        [-0.2073,  0.2859, -0.4720,  0.1073, -0.4903, -0.4741,  0.5049,  0.0941,
          0.4610,  0.3093, -0.4693, -0.4578,  0.0966, -0.2094, -0.3188,  0.1935,
          0.2300,  0.0417,  0.0900, -0.2882]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.0000, 0.2466, 0.0000, 0.1987, 0.0000, 0.0000, 0.4755, 0.0041, 0.3738,
         0.0953, 0.0000, 0.0000, 0.0259, 0.0000, 0.0000, 0.1725, 0.2422, 0.0000,
         0.2988, 0.0000],
        [0.0000, 0.3139, 0.0000, 0.2604, 0.0000, 0.0000, 0.5266, 0.1123, 0.6165,
         0.1251, 0.0000, 0.0000, 0.1692, 0.0000, 0.00

In [11]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3, 28, 28)
logits = seq_modules(input_image)

In [12]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

## Model Parameters

In [13]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values: {param[:2]} \n")

Model structure: NerualNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values: tensor([[-0.0276,  0.0274, -0.0060,  ..., -0.0081, -0.0246,  0.0256],
        [ 0.0341,  0.0147,  0.0260,  ...,  0.0304,  0.0223,  0.0284]],
       device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values: tensor([ 0.0273, -0.0245], device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values: tensor([[ 0.0013, -0.0437,  0.0262,  ..., -0.0118,  0.0224,  0.0302],
        [ 0.0330, -0.0379, -0.0196,  ..., -0.0410,  0.0056, -0.0210]],
       device='cuda:0', grad_fn=<Slice